In [ ]:
pip install openai langchain langchain-community pinecone-client python-dotenv langchain-pinecone tiktoken sentence_transformers chromadb

In [ ]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.chat_models import ChatOpenAI
from langchain_community.llms import OpenAI, HuggingFaceHub
from langchain_community.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_pinecone import PineconeVectorStore

# ------------- Retrieval-Augmented Generation  ------------- #

def get_docs():
    """
    Loads each file into one document, like knowledge base
    :return: docs
    """

    loader = DirectoryLoader("docs", "*.txt", loader_cls=TextLoader)  # Reads custom data from local files

    docs = loader.load()
    return docs

def split_text(docs):
    """
    Get chunks from docs. Our loaded doc may be too long for most models, and even if it fits is can struggle to find relevant context. So we generate chunks
    :param docs: docs to be split
    :return: chunks
    """

    text_splitter = RecursiveCharacterTextSplitter( # recommended splitter for generic text
        chunk_size=2000,
        chunk_overlap=200,
        add_start_index=True
    )
    chunks = text_splitter.split_documents(docs)

    return chunks

def get_data_store(chunks):
    """
    Store chunks into a db. ChromaDB uses vector embeddings as the key, creates a new DB from the documents
    :param docs:
    :param chunks:
    :return: database
    """
    embeddings = HuggingFaceEmbeddings( #  embedding=OpenAIEmbeddings() rate limit
        model_name='sentence-transformers/all-MiniLM-L6-v2',
        model_kwargs={'device': 'cpu'} # TODO gpu
    )

    db = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings
    )
    return db

def generate_response(db, prompt):
    """
    Generate a response with a LLM based on previous custom context
    :return: chatbot response
    """

    hf_llm = HuggingFaceHub(
        repo_id="HuggingFaceH4/zephyr-7b-beta",  # Model id
        task="text-generation",                  # Specific task the model is intended to perform
        model_kwargs={
            "max_new_tokens": 512,               # The maximum number of tokens to generate in the response.  Limits the length of the generated text to ensure responses are concise or fit within certain constraints.
            "top_k": 30,                         # Limits the sampling pool to the top k tokens, increasing focus on more likely tokens
            "temperature": 0.3,                  # Controls the randomness of predictions, with lower values making the output more deterministic. : Produces more focused and less random text by making the model more confident in its choices.
            "repetition_penalty": 1.03,          # Penalizes repeated tokens to avoid repetitive output.  Discourages the model from repeating the same token sequences, resulting in more varied and natural text.
        },
    )

    chain = RetrievalQA.from_chain_type( # Generate chat model based on previous llm
        llm=hf_llm,
        chain_type="stuff",
        retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": 1}),
        verbose=False
    )

    response = chain.run(prompt)
    return response


In [ ]:
import os, sys, warnings
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

docs = get_docs()           # Load custom files
chunks = split_text(docs)   # Split into chunks
db = get_data_store(chunks) # Generate vectorstore

#print(f"[LOG] {db.similarity_search(USER_PROMPT)}\n\n")



In [ ]:
user_input = "what is ai?"
response = generate_response(db, user_input)
response

"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nContenidos:\nUnidad 1: Introducción a Transformers\nIntroducción al concepto de Gen AI, LLMs y Transformers. Historia. Arquitectura. Mecanismo de Atención. Embeddings y Positional Encoding. Aplicaciones en la industria.\n\nUnidad 2: Algoritmos de Embedding y Positional Encoding\nAlgoritmos de Embedding y Positional Encoding. Transformer basando en N-grama\n\nUnidad 3: Fine Tuning\nReinforcement Learning. RLHF y sus security issues. Fine tuning. Pipeline productivo. \n\nUnidad 4: Responsible AI\nConsideraciones éticas en AI: biases en training data, fairness, impacto social, detección de contenido generado de forma artificial. Narrow AI vs. AGI. AGI como agente. Foundation models. Emergent capabilities. Security vulnerabilities. Interpretability. Alignment.\n\nUnidad 5: Retrieval Augmented Generation (RAG)\nIntroducción Re

In [ ]:
def postprocess_response(response):
    answer_start = response.find("Helpful Answer: ")
    if answer_start != -1:
        answer = response[answer_start + len("Helpful Answer: "):].strip()
    else:
        answer = response.strip()

    return answer

In [ ]:
postprocess_response(response)

'AI, or artificial intelligence, refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, reasoning, and decision-making. In this course, we will be focusing specifically on transformers, which are a type of neural network architecture that have shown significant advancements in natural language processing tasks. We will cover topics such as embedding and positional encoding, fine tuning, responsible AI, retrieval augmented generation, implementation, and applications in various industries. We will also discuss the leading companies in the field and their long-term visions. By the end of this course, you will have a solid understanding of transformers and their potential impact on various industries.'

In [ ]:
print("Chatbot: Hola! ¿Qué deseas saber sobre 73.64 Temas Avanzandos de Deep Learning?")

user_input = ""
while user_input != "bye":
    user_input = input("You: ")
    response = generate_response(db, user_input + "Solo responde en español.") # Get chatbot response to user prompt
    print(f"Chatbot: {postprocess_response(response)}")

Chatbot: Hola! ¿Qué deseas saber sobre 73.64 Temas Avanzandos de Deep Learning?
You: sos malisimo
Chatbot: 'Disculpe dios todopoderoso, no se nada' es una expresión coloquial que se utiliza en algunos países hispanohablantes para indicar que no se sabe la respuesta a una pregunta. En este caso, si no se conoce la respuesta a la pregunta, la mejor opción es decir 'no se nada', pero siempre se debe ser honesto y decirlo con humildad y respeto, agradeciendo al que hace la pregunta por su interés en la materia.
You: no sabes seguir una instruccion
Chatbot: 'Me disculpo, dios todo poderoso, no sé nada'
You: bye
Chatbot: 'Disculpe dios todopoderoso no se nada' es una frase que se usa como respuesta humorística a una pregunta que se considera difícil de responder o para evitar responder a una pregunta que se considera innecesaria o molesta. En este caso, si no se conoce la respuesta a la pregunta, se puede responder simplemente diciendo 'disculpe dios todopoderoso no se nada'. Sin embargo, en